In [ ]:
import numpy as np
import pandas as pd

import sklearn

import tensorflow as tf
from tensorflow import keras
from keras import layers, models, optimizers
from keras.regularizers import L1L2

import datawig

import os
import time
import gc


In [ ]:
train_X = pd.read_csv('./train_X.csv')
test_X = pd.read_csv('./test_X.csv')

train_y = pd.read_csv('./train_y.csv')

sub = pd.read_csv('./data/sample_submission.csv')

In [ ]:
cc = train_X.columns.sort_values()

rain_X = train_X[cc]
test_X = test_X[cc]

In [4]:
target = (train_y - np.mean(train_y))/np.std(train_y)

In [23]:
n_layers = 256
n_node = 3

inputs = layers.Input(shape =(train_X.shape[1], ))

x = layers.Dense(n_layers)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

for i in range(n_node):
    x = layers.Dense(n_layers//(2**(i+1)), kernel_regularizer=L1L2(l2=0.001), kernel_initializer='he_normal')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.1)(x)

outputs = layers.Dense(1, activation='linear')(x)

m = models.Model(inputs, outputs)

m.compile(optimizer = 'adam',
            loss = 'mse')

In [24]:
m.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 275)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               70656     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0   

In [25]:
es = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [26]:
m.fit(train_X, target, 
      epochs = 1000,
     validation_split=0.3,
     callbacks = [es])

Train on 1021 samples, validate on 439 samples
Epoch 1/1000
1021/1021 [==============================] - 1s 1ms/step - loss: 1.5352 - val_loss: 0.9014
Epoch 2/1000
1021/1021 [==============================] - 0s 275us/step - loss: 0.8754 - val_loss: 0.8508
Epoch 3/1000
1021/1021 [==============================] - 0s 274us/step - loss: 0.8593 - val_loss: 0.7840
Epoch 4/1000
1021/1021 [==============================] - 0s 268us/step - loss: 0.7817 - val_loss: 0.7604
Epoch 5/1000
1021/1021 [==============================] - 0s 268us/step - loss: 0.7512 - val_loss: 0.7273
Epoch 6/1000
1021/1021 [==============================] - 0s 270us/step - loss: 0.6970 - val_loss: 0.7248
Epoch 7/1000
1021/1021 [==============================] - 0s 270us/step - loss: 0.6938 - val_loss: 0.7539
Epoch 8/1000
1021/1021 [==============================] - 0s 278us/step - loss: 0.6234 - val_loss: 0.6976
Epoch 9/1000
1021/1021 [==============================] - 0s 269us/step - loss: 0.6495 - val_loss: 0.6560
E

1021/1021 [==============================] - 0s 267us/step - loss: 0.1830 - val_loss: 0.3652
Epoch 78/1000
1021/1021 [==============================] - 0s 281us/step - loss: 0.1953 - val_loss: 0.3637
Epoch 79/1000
1021/1021 [==============================] - 0s 272us/step - loss: 0.1846 - val_loss: 0.3643
Epoch 80/1000
1021/1021 [==============================] - 0s 277us/step - loss: 0.1996 - val_loss: 0.4723
Epoch 81/1000
1021/1021 [==============================] - 0s 267us/step - loss: 0.1793 - val_loss: 0.3602
Epoch 82/1000
1021/1021 [==============================] - 0s 280us/step - loss: 0.1811 - val_loss: 0.3975
Epoch 83/1000
1021/1021 [==============================] - 0s 271us/step - loss: 0.1747 - val_loss: 0.3364
Epoch 84/1000
1021/1021 [==============================] - 0s 279us/step - loss: 0.1579 - val_loss: 0.3396
Epoch 85/1000
1021/1021 [==============================] - 0s 271us/step - loss: 0.1800 - val_loss: 0.3562
Epoch 86/1000
1021/1021 [==========================

1021/1021 [==============================] - 0s 273us/step - loss: 0.0835 - val_loss: 0.2778
Epoch 154/1000
1021/1021 [==============================] - 0s 270us/step - loss: 0.0802 - val_loss: 0.2834
Epoch 155/1000
1021/1021 [==============================] - 0s 282us/step - loss: 0.0911 - val_loss: 0.2864
Epoch 156/1000
1021/1021 [==============================] - 0s 272us/step - loss: 0.0777 - val_loss: 0.2986
Epoch 157/1000
1021/1021 [==============================] - 0s 267us/step - loss: 0.0782 - val_loss: 0.2670
Epoch 158/1000
1021/1021 [==============================] - 0s 268us/step - loss: 0.0835 - val_loss: 0.2522
Epoch 159/1000
1021/1021 [==============================] - 0s 272us/step - loss: 0.0744 - val_loss: 0.2476
Epoch 160/1000
1021/1021 [==============================] - 0s 277us/step - loss: 0.0861 - val_loss: 0.2632
Epoch 161/1000
1021/1021 [==============================] - 0s 283us/step - loss: 0.0943 - val_loss: 0.2584
Epoch 162/1000
1021/1021 [=================

In [27]:
k = m.predict(test_X)

In [28]:
ans = (k * np.std(train_y)[0]) + np.mean(train_y)[0]

In [29]:
sub['SalePrice'] = ans

In [30]:
sub

,Id,SalePrice
0,1461,138737.906250
1,1462,107663.804688
2,1463,157574.984375
3,1464,160758.687500
4,1465,155979.546875
...,...,...
1454,2915,226571.421875
1455,2916,185687.859375
1456,2917,144073.437500
1457,2918,165054.062500


In [31]:
sub.to_csv('./sub/sub13.csv', index=False)